# Jimmy Wrangler, Data Explorer

Dalton Hahn (2762306)

## Donald Trump Approval Ratings (Job approval rating and trendline)
https://projects.fivethirtyeight.com/trump-approval-data/approval_polllist.csv
https://projects.fivethirtyeight.com/trump-approval-data/approval_topline.csv

## Exploratory Data Analysis
My idea is to examine the three datasets to see if there is any correlation or connection between President Trump's tweets and the lawsuits he and his company have been involved in.  Additionally, I would like to see if there is any correlation or connection between his tweets and his overall approval rating.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import math
from statistics import mean, stdev

## Read in the Data

In [2]:
df_top = pd.read_csv("../data/external/approval_topline.csv")
df_poll = pd.read_csv("../data/external/approval_polllist.csv")

In [3]:
df_top.head(1)

,president,subgroup,modeldate,approve_estimate,approve_hi,approve_lo,disapprove_estimate,disapprove_hi,disapprove_lo,timestamp
0,Donald Trump,Voters,9/10/2019,42.38458,46.79176,37.9774,53.32431,56.70696,49.94167,09:39:21 10 Sep 2019


In [4]:
df_poll.head(1)

,president,subgroup,modeldate,startdate,enddate,pollster,grade,samplesize,population,weight,...,disapprove,adjusted_approve,adjusted_disapprove,multiversions,tracking,url,poll_id,question_id,createddate,timestamp
0,Donald Trump,All polls,9/10/2019,1/20/2017,1/22/2017,Gallup,B,1500.0,a,0.245429,...,45.0,46.13177,43.29169,NaN,T,http://www.gallup.com/poll/201617/gallup-daily...,49253,77265,1/23/2017,09:37:40 10 Sep 2019
